In [ ]:
#test synthetic damagelevel vs real
import numpy as np
from tabulate import tabulate
from UNET import unet
from raster import raster
import os
import shutil
import albumentations as A
from syntheticdamage import syntheticspall, syntheticflor
import cv2 as cv
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from syntheticwall import syntheticwall
import random
from addnoise import addperlin1, addperlin2
import os

os.chdir("../data")

#path1 = 'C:/Users/eejmws/OneDrive - University of Leeds/Incoming data/'
#path2 = 'C:/Users/jackm/OneDrive - University of Leeds/Incoming data/'
 
#isdir1 = os.path.isdir(path1) 
#isdir2 = os.path.isdir(path2) 

#if isdir1 == True:
#    os.chdir(path1)
#elif isdir2 == True:
#    os.chdir(path2)
#else:
#    print("directory not found")






saveloc = ['testsynth0/','testsynth0.1/''testsynth1/','testsynth2/','testsynth3/','testsynth4/']#,'testideal4/','testideal5/']
testmaskfile = "testmask.tiff"
testfile = "test.tiff"
#cloud ="synthetic_defect_wall.tiff"
#maskfile = "mask.png"
cloud ="traincloud.las"
maskfile = "mask.png"
#reslist = [0.7, 1, 1.5, 2, 3, 4]
damagelist = [0.7]#, 0.55, 0.7, 0.75, 0.8]

damagelevel = damagelist[0]

testim = Image.open("test.tiff")
wallim = Image.open("wall.tiff")



res1 = 2
dimlist = [256]
dim1 = dimlist[0]
synthproplist = [0,0.1,0.2, 0.5,1,2]
table = [['Network', 'encoder', 'epochs', 'IOU', 'Precision', 'recall','dim']]
#for dim1, saveloc in zip(dimlist,saveloc):
variationmagnitude = 0.3
x = 30

try:
    os.makedirs("synth/")
except OSError:
    pass 



for n in range(0,x):

    t = 1
    while t == 1:

        try:
            blocklength = random.randint(50,350)
            blockheight = int(max(0.5,random.random())*blocklength/1.3)
            #create wall
            synthwall,synthmask = syntheticwall(int(wallim.size[0]/2),int(wallim.size[1]/2),blockheight,blocklength,"syntheticwalltest"+str(n)+".tiff")
            synthspalledmask = synthmask
            #add spalling
            synthspalledwall3 = syntheticspall(synthwall,synthmask,damagelevel)
            #add efflorescence
            synthspalledwall2 = syntheticflor(synthspalledwall3,synthmask,damagelevel)
            #surface variation
            synthspalledwall1 = addperlin1(synthspalledwall2,variationmagnitude)
            #wall deformation
            synthspalledwall = addperlin2(synthspalledwall1,variationmagnitude*6)

            spalledwallim = Image.fromarray(synthspalledwall) 
            spalledwallim.save("synth/"+"synthetic_defect_wall"+str(n)+".tiff")
            spalledmaskim = Image.fromarray(synthspalledmask*255) 
            spalledmaskim.save("synth/"+"synthetic_defect_mask"+str(n)+".png")
            t = 0
        except:
            continue




    



for synthprop, saveloc in zip(synthproplist,saveloc):
    
   #  testim = Image.open("test.tiff")
   #  wallim = Image.open("wall.tiff")
   #  #maskim = Image.open("wall.tiff")
   #  wall = np.array(wallim)
   #  #mask = np.array(maskim)
   #  mask = wall>0.9
   #  test = np.array(testim)
   # #damagelevel = 0.7
   #  spalledwall,spallingmask = syntheticspall(wall,mask,damagelevel)
   #  spalledwallim = Image.fromarray(spalledwall) 
   #  spalledwallim.save("synthetic_defect_wall.tiff")
    
    try:
        os.makedirs(saveloc)
        os.makedirs(saveloc+"results/")
        
        os.makedirs("synth/"+saveloc)
        os.makedirs("synth/"+saveloc+"results/")
        
    except OSError:
        pass  
    
    
    shutil.copyfile(testfile, saveloc+testfile)
    shutil.copyfile(testmaskfile, saveloc+testmaskfile)
    shutil.copyfile(cloud, saveloc+cloud)
    shutil.copyfile(maskfile, saveloc+maskfile)
    raster(cloud,saveloc,mask = 0,res=res1,dim=dim1) 
    raster(maskfile,saveloc,mask = 1,res=res1, dim = dim1)
    
    

    
    nosamples = len(os.listdir(saveloc+"croppedimages/"))
    nosynthtoadd = int(nosamples*synthprop)
    count = 0
    nosynthadded = 1
    while nosynthadded <=nosynthtoadd:
        print(count)
        
        synthtoadd = "synthetic_defect_wall"+str(count)+".tiff"
        masktoadd = "synthetic_defect_mask"+str(count)+".png"
        # for filename in os.listdir("synth"saveloc):
        # filepath = os.path.join(dirpath, filename)
        # try:
        #     shutil.rmtree(filepath)
        # except OSError:
        #     os.remove(filepath)


        os.makedirs("synth/"+saveloc+str(count)+"/")
        raster("synth/"+synthtoadd,"synth/"+saveloc+str(count)+"/",mask = 0,res=res1,dim=dim1)
        raster("synth/"+masktoadd,"synth/"+saveloc+str(count)+"/",mask = 1,res=res1, dim = dim1)
        
        nosynthsamples = len(os.listdir("synth/"+saveloc+str(count)+"/"+"croppedimages/"))
        
        for a in range(1, nosynthsamples+1):
            shutil.copyfile("synth/"+saveloc+str(count)+"/"+"croppedimages/"+str(a)+".tiff", saveloc+"croppedimages/"+str(nosamples+nosynthadded)+".tiff")
            shutil.copyfile("synth/"+saveloc+str(count)+"/"+"croppedmasks/"+str(a)+".tiff", saveloc+"croppedmasks/"+str(nosamples+nosynthadded)+".tiff")

            nosynthadded =nosynthadded + 1
            
        count = count+1
        
        
        
        
        
    
    
    encoderstotest = ['mobilenet_v2']
    networkstotest = ['Unet']#,'UnetPlusPlus','DeepLabV3']
    test = 1
    n = 0
    
    
    transparams = [
       # A.RandomSizedCrop(min_max_height=(64, 512), height=512, width=512, w2h_ratio=1.0, interpolation=1, always_apply=False, p=0.75),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
       # A.RandomBrightnessContrast(p=0.7),
        A.GaussNoise(always_apply=False, p=0.3, var_limit=(3, 9)),
      #  A.RandomContrast(always_apply=False, p=0.3, limit=(-0.6, 0.6)),
       # A.RandomRotate90(always_apply=False, p=0.75)
       # A.CLAHE(p=1),

    ]
    transparamsv = [
       # A.RandomSizedCrop(min_max_height=(64, 512), height=512, width=512, w2h_ratio=1.0, interpolation=1, always_apply=False, p=0.75),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
       # A.RandomBrightnessContrast(p=0.7),
        A.GaussNoise(always_apply=False, p=0.3, var_limit=(5, 5)),
      #  A.RandomContrast(always_apply=False, p=0.3, limit=(-0.6, 0.6)),
       # A.RandomRotate90(always_apply=False, p=0.75)
       # A.CLAHE(p=1),

    ]
    
    for network in networkstotest:
        for encoder in encoderstotest:
            pretrain = "imagenet"
            if dim1 >= 512:
                 batch = 2
            else:
                 batch = 4
            path =saveloc
            inno=1
            nepochs = 50
            print(network)
            print(encoder)
            #try:
            iou_score,precision,recall,epochsf = unet(test, inno, batch, path, nepochs, network, encoder, pretrain, dim1, transparams, transparamsv)
            print(iou_score)
            table.append([network, encoder, epochsf, iou_score, precision, recall, dim1])
            #except:
            print('training failed')#
            
print(tabulate(table))

damage added
damage added
damage added
damage added
damage added
damage added
damage added
floresced
damage added
damage added
damage added
floresced
damage added
damage added
floresced


In [ ]:
os.listdir("synth/")

In [ ]:
import cv2 as cv
from PIL import Image

im = Image.open("synth/synthetic_defect_wall10.tiff")
imnp = np.array(im)
#im = cv.imread("synth/synthetic_defect_wall4.tiff")
plt.imshow(imnp)

In [ ]:
im = Image.open("test.tiff")
imnp = np.array(im)
#im = cv.imread("synth/synthetic_defect_wall4.tiff")
plt.imshow(imnp)

In [ ]:
#im = imread("C:/Users/eejmws/OneDrive - University of Leeds/Pytorch/data/testsynth2/croppedimages/83.tiff")
from skimage.io import imread
im = imread("C:/Users/eejmws/OneDrive - University of Leeds/Pytorch/data/testsynth2/croppedimages/125.tiff")

In [ ]:
a =np.array(im)

In [ ]:
plt.imshow(a)

In [ ]:
im = Image.open("C:/Users/eejmws/OneDrive - University of Leeds/Pytorch/data/testsynth2/croppedimages/86.tiff")
imnp = np.array(im)
#im = cv.imread("synth/synthetic_defect_wall4.tiff")
plt.imshow(imnp)